# Glassdoor Salaries

## Create a model from scratch in Keras

X

## Model setup

We need to import all the modules we'll be using from numpy, scipy, and keras:

In [16]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.io import mmread
import numpy as np
import math
import csv

import keras
import json
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [44]:
%pwd
np.bool

bool

Load Data


In [73]:
data_dir = '/Users/phillip/Data/salary-estimates'
sparse_training_matrix = mmread('%s/training-matrix-test.txt' % data_dir)
print "Done"

Done


- sparse matrix text = training row + " " + column (where value is +1)

In [58]:
from scipy.sparse import coo_matrix
print sparse_training_matrix.shape
print sparse_training_matrix.getrow(568370)
print coo_matrix(sparse_training_matrix.getrow(568370), dtype=np.bool)
print "Done"
#classes = [class_dict[str(i)][1] for i in range(len(class_dict))]
#classes[:12]
# a.todense() or a.M - Return a dense matrix representation of this matrix. (numpy.matrix)
# a.tosparse_training_matrix = mmread('%s/training-matrix.txt' % data_dir)array() or a.A - Return a dense ndarray representation of this matrix. (numpy.array)

(6516817, 1812571)
  (0, 0)	1.0
  (0, 0)	True
Done


In [69]:
def salaryToTarget(salary):
    return int(round((max((min((salary * 1.0, 600000.0)), 15000))-15000) / 5000, 0))

def targetToSalary(target):
    return (target * 5000) + 15000

def logSalaryToTarget(logSalary):
    return salaryToTarget(math.pow(math.e, logSalary))

print salaryToTarget(15)
print targetToSalary(8)
print logSalaryToTarget(10.3089859934221)

salaries = np.zeros(6523000, dtype=np.int)

with open('%s/log-salaries-test-truncated.csv' % data_dir,'r') as dest_f:
    data_iter = csv.reader(dest_f)
    for row in data_iter:
        salaries[int(row[0])] = logSalaryToTarget(float(row[1]))
    
print "Created salaries"
    
#targets = mmread('%s/training-matrix.txt' % data_dir)

0
55000
3


## Model creation

### Features

- Binary vector of length 1812571 

### Training Data

- Log salary
- Salaries: $15K => $600K
- Divide into classes of $5K ranges => 117 classes

### Targets

- 117 length vector


In [70]:
def ConvBlock(layers, model, filters):
    for i in range(layers): 
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

...and here's the fully-connected definition.

In [71]:
def FCBlock(model):
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

Define the model

In [72]:
def Phillip_Salary_1():
    model = Sequential()
    
    model.add(Dense(512, input_shape=(1812571,)))
    ConvBlock(3, model, 128)

    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(117, activation='softmax'))
    return model

We'll learn about what these different blocks do later in the course. For now, it's enough to know that:

- Convolution layers are for finding patterns in images
- Dense (fully connected) layers are for combining patterns across an image

Now that we've defined the architecture, we can create the model like any python object:

In [29]:
model = Phillip_Salary_1()

As well as the architecture, we need the weights that the VGG creators trained. The weights are the part of the model that is learnt from the data, whereas the architecture is pre-defined based on the nature of the problem. 

Downloading pre-trained weights is much preferred to training the model ourselves, since otherwise we would have to download the entire Imagenet archive, and train the model for many days! It's very helpful when researchers release their weights, as they did here.